In [20]:
from geopy.geocoders import Nominatim
from meteostat import Point, Daily
from datetime import datetime, timedelta
import pytz

def get_lat_lon(city_name):
    geolocator = Nominatim(user_agent="weather_app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError(f"위치를 찾을 수 없습니다: {city_name}")

def weather_info(city):
    try:
        lat, lon = get_lat_lon(city)
    except ValueError as e:
        print(e)
        return None

    # Create Point for the city
    location = Point(lat, lon)

    # Set timezone to KST (Korean Standard Time)
    kst = pytz.timezone('Asia/Seoul')

    # Get today's and yesterday's date in KST
    today = datetime.now(kst).date()
    yesterday = today - timedelta(days=1)

    # Get data for yesterday and today in UTC
    start = datetime.combine(yesterday, datetime.min.time())
    end = datetime.combine(today, datetime.min.time()) + timedelta(days=1)

    # Get daily data for the time period
    data = Daily(location, start, end)
    data = data.fetch()

    # Check if data is available
    if data.empty:
        print("데이터를 가져오지 못했습니다. 날짜 범위 또는 위치를 확인하세요.")
        return None

    result = {}

    if len(data) >= 1:
        yesterday_data = data.iloc[0]
        result['yesterday'] = {
            "avg_temp": yesterday_data['tavg'],
            "max_temp": yesterday_data['tmax'],
            "min_temp": yesterday_data['tmin'],
            "precipitation": yesterday_data['prcp'],
            "wind_speed": yesterday_data['wspd']
        }

    if len(data) >= 2:
        today_data = data.iloc[1]
        result['today'] = {
            "avg_temp": today_data['tavg'],
            "max_temp": today_data['tmax'],
            "min_temp": today_data['tmin'],
            "precipitation": today_data['prcp'],
            "wind_speed": today_data['wspd']
        }

    if 'yesterday' in result and 'today' in result:
        result['comparison'] = {
            "temp_max_difference": result['today']['max_temp'] - result['yesterday']['max_temp'],
            "temp_min_difference": result['today']['min_temp'] - result['yesterday']['min_temp'],
            "temp_difference": result['today']['avg_temp'] - result['yesterday']['avg_temp']
        }

    return result


In [22]:
from openai import OpenAI
import json
import os
import time

# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI(api_key = "") # API 키!
ASSISTANT_ID = 'asst_tUcre9CS6mNDuLm90VEahIZU'

def wait_run(client, run, thread):
  while True:
    run_check = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )
    print(run_check.status)
    if run_check.status in ['queued','in_progress']:
      time.sleep(2)
    else:
      break
  return run_check


# 사용자로부터 도시 이름을 입력받습니다.
city = input("날씨 정보를 확인할 도시 이름을 입력하세요: ")

# 날씨 정보 조회
weather_data = weather_info(city)
if not weather_data:
    print("날씨 정보를 가져오지 못했습니다.")
else:
    # OpenAI 스레드 생성 및 메시지 전송
    thread = client.beta.threads.create(
        messages=[
            {
                "role" : "user",
                "content" : f"Please provide clothing recommendations for the weather in {city} in JSON format."
            }
        ]
    )
    
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=ASSISTANT_ID
    )

    run_check = wait_run(client, run, thread)
    run_check.status
    # 결과: requires_action

    tool_calls = run_check.required_action.submit_tool_outputs.tool_calls
    tool_outputs = []
    for tool in tool_calls:
        func_name = tool.function.name
        kwargs = json.loads(tool.function.arguments)
        output = locals()[func_name](**kwargs)
        tool_outputs.append(
            {
            "tool_call_id": tool.id,
            "output": str(output)
            }
        )

    if tool_outputs:
        try:
            run = client.beta.threads.runs.submit_tool_outputs_and_poll(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
            run_check = wait_run(client,run,thread)
            run_check.status    
            # 결과: completed    
        except Exception as e:
            print(e)
    else:
        print("No tool outputs to submit.")
   

    if run.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        for msg in reversed(messages.data):
            print(f"{msg.role}: {msg.content[0].text.value}")
        else:
            print(run.status)

in_progress
requires_action


TypeError: weather_info() got an unexpected keyword argument 'yesterday'

In [25]:
import openai
import json
from geopy.geocoders import Nominatim
from meteostat import Point, Daily
from datetime import datetime, timedelta
import pytz

client = openai.OpenAI(api_key="")

def get_lat_lon(city_name):
    geolocator = Nominatim(user_agent="weather_app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError(f"위치를 찾을 수 없습니다: {city_name}")

def get_weather_info(city):
    try:
        lat, lon = get_lat_lon(city)
    except ValueError as e:
        print(e)
        return None

    location = Point(lat, lon)
    kst = pytz.timezone('Asia/Seoul')
    today = datetime.now(kst).date()
    yesterday = today - timedelta(days=1)
    start = datetime.combine(yesterday, datetime.min.time())
    end = datetime.combine(today, datetime.min.time()) + timedelta(days=1)

    data = Daily(location, start, end)
    data = data.fetch()

    if data.empty:
        print("데이터를 가져오지 못했습니다. 날짜 범위 또는 위치를 확인하세요.")
        return None

    yesterday_data = data.iloc[0] if len(data) >= 1 else None
    today_data = data.iloc[1] if len(data) >= 2 else None

    return json.dumps({
        "yesterday": {
            "avg_temp": yesterday_data['tavg'],
            "max_temp": yesterday_data['tmax'],
            "min_temp": yesterday_data['tmin'],
            "precipitation": yesterday_data['prcp'],
            "wind_speed": yesterday_data['wspd']
        },
        "today": {
            "avg_temp": today_data['tavg'],
            "max_temp": today_data['tmax'],
            "min_temp": today_data['tmin'],
            "precipitation": today_data['prcp'],
            "wind_speed": today_data['wspd'],
            "temp_difference": today_data['tmax'] - today_data['tmin']
        },
        "comparison": {
            "temp_max_difference": today_data['tmax'] - yesterday_data['tmax'],
            "temp_min_difference": today_data['tmin'] - yesterday_data['tmin']
        }
    })


assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to recommend clothing based on yesterday's and today's weather.",
    model="gpt-4o",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_weather_info",
                "description": "Get yesterday and today's weather data for clothing recommendations",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "The name of the city to get weather information for"
                        }
                    },
                    "required": ["city"]
                }
            }
        }
    ]
)

city = input("날씨 정보를 확인할 도시 이름을 입력하세요: ")

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"Can you provide clothing recommendations based on yesterday's and today's weather in {city}?",
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

if run.status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(messages)
else:
    print(run.status)

tool_outputs = []

for tool in run.required_action.submit_tool_outputs.tool_calls:
    if tool.function.name == "get_weather_info":
        function_args = json.loads(tool.function.arguments)
        function_response = get_weather_info(function_args["city"])
        tool_outputs.append({
            "tool_call_id": tool.id,
            "output": function_response
        })

if tool_outputs:
    try:
        run = client.beta.threads.runs.submit_tool_outputs_and_poll(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
        print("Tool outputs submitted successfully.")
    except Exception as e:
        print("Failed to submit tool outputs:", e)
else:
    print("No tool outputs to submit.")

if run.status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    for message in messages:
        print(message['content'])
else:
    print(run.status)

requires_action
Tool outputs submitted successfully.


TypeError: 'Message' object is not subscriptable

In [13]:

run = openai.beta.threads.runs.retrieve(
  thread_id=thread.id,  # 생성한 스레드 ID
  run_id=run_id
)

if run.status == 'completed':
  messages = openai.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)


requires_action


In [14]:
# Define the list to store tool outputs
tool_outputs = []
 
# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "weather_info":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": "57"
    })
 
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = openai.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
  except Exception as e:
    print(e)
else:
  print("No tool outputs to submit.")
 
if run.status == 'completed':
  messages = openai.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[1].content[0].text.value)
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)

나는 서울 용산구에 있어, 뭐를 입어야 할까?
서울 용산구의 오늘 날씨를 알려줄게.

어제와 비교해보면:
- 오늘은 어제보다 더 덥습니다.
- 오늘 기온이 30도 이상이라 엄청 덥습니다.

오늘 추천하는 옷차림:
- 가벼운 반팔 옷이나 시원한 옷을 입는 것이 좋습니다.
- 일교차가 크지 않지만, 낮 기온이 높아 더위를 많이 탈 수 있으니 주의하세요.

더운 날 건강 관리 잘하시고 시원하게 보내세요!
